In [7]:
from collections import Counter, defaultdict
from copy import deepcopy
from joblib import Parallel, delayed
from time import time
from tqdm import tqdm
import enum
import itertools
import pandas as pd

def timeit(method):
    def timed(*args, **kw):
        ts = time()
        result = method(*args, **kw)
        te = time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = (te - ts)
        else:
            print '%r \t\t %.4f s' % \
                  (method.__name__, (te - ts))
        return result
    return timed

In [8]:
datafile = 'data/ay1819-usp.csv'

@timeit
def get_data():
    rawdata = pd.read_csv(datafile)
    data = rawdata[['Gender Description', 'Room Preference Description 1', 'Room Preference Description 2', 'Room Preference Description 3', 'Floor Gender Preference']]
    data = data\
        .rename(columns = {
            'Gender Description': 'gender',
            'Room Preference Description 1': 'room_pref_1',
            'Room Preference Description 2': 'room_pref_2',
            'Room Preference Description 3': 'room_pref_3',
            'Floor Gender Preference': 'gender_pref',
        })\
        .replace('USP, Single (Corridor, Air-Con)', 'corri_air')\
        .replace('USP, Single (6 bdrm Apt, Air-Con)', 'suite_air')\
        .replace('USP, Single (Corridor, Non Air-Con)', 'corri_non')\
        .replace('USP, Single (6 bdrm Apt, Non Air-Con)', 'suite_non')\
        .replace('Mixed Gender Floor', 'mixed')\
        .replace('No Preference', 'mixed')\
        .replace('Single Gender Floor', 'single')\
        .fillna('no_pref')
    return data

data = get_data()

'get_data' 		 0.0204 s


In [9]:
# Define floors
floors = set(range(4, 22))
male_floors = set([8, 10, 16, 19])
female_floors = set([7, 14, 18, 20])
mixed_floors = floors.difference(male_floors).difference(female_floors)
rf_floors = set(range(5, 22, 4))
laundry_floors = set([9, 17])
only_rf_floors = rf_floors.difference(laundry_floors)
normal_floors = floors.difference(rf_floors)
freshmen_floors = set([6, 8, 11, 14, 16, 17, 20])
senior_floors = floors.difference(freshmen_floors)

In [10]:
@timeit
def get_num_floor_types_iterator():
    rf_floors_consider = only_rf_floors.difference(freshmen_floors)
    laundry_floors_consider = laundry_floors.difference(freshmen_floors)
    normal_floors_consider = normal_floors.difference(freshmen_floors)
    
    for male_rf in range(len(rf_floors_consider) + 1):
        for female_rf in range(len(rf_floors_consider) + 1 - male_rf):
            mixed_rf = len(rf_floors_consider) - male_rf - female_rf
            
            for male_laundry in range(len(laundry_floors_consider) + 1):
                for female_laundry in range(len(laundry_floors_consider) + 1 - male_laundry):
                    mixed_laundry = len(laundry_floors_consider) - male_laundry - female_laundry
            
                    for male_normal in range(len(normal_floors_consider) + 1):
                        for female_normal in range(len(normal_floors_consider) + 1 - male_normal):
                            mixed_normal = len(normal_floors_consider) - male_normal - female_normal
  
                            combination = dict()
                            combination['Male'] = {'rf': male_rf, 'laundry': male_laundry, 'normal': male_normal}
                            combination['Female'] = {'rf': female_rf, 'laundry': female_laundry, 'normal': female_normal}
                            combination['Mixed'] = {'rf': mixed_rf, 'laundry': mixed_laundry, 'normal': mixed_normal}
                            yield combination

In [11]:
class RoomType(enum.Enum):
    MALE_CORRI_AIR = 0
    MALE_CORRI_NON = 1
    MALE_SUITE_AIR = 2
    MALE_SUITE_NON = 3
    FEMALE_CORRI_AIR = 4
    FEMALE_CORRI_NON = 5
    FEMALE_SUITE_AIR = 6
    FEMALE_SUITE_NON = 7
    MIXED_CORRI_AIR = 8
    MIXED_CORRI_NON = 9
    MIXED_FEMALE_SUITE_AIR = 10
    MIXED_FEMALE_SUITE_NON = 11
    MIXED_MALE_SUITE_AIR = 12
    MIXED_MALE_SUITE_NON = 13
    NO_ROOM = 14
    
room_pref_to_type_map = {
    'Female': {
        'suite_air': RoomType.FEMALE_SUITE_AIR,
        'suite_non': RoomType.FEMALE_SUITE_NON,
        'corri_air': RoomType.FEMALE_CORRI_AIR,
        'corri_non': RoomType.FEMALE_CORRI_NON,
    },
    'Male': {
        'suite_air': RoomType.MALE_SUITE_AIR,
        'suite_non': RoomType.MALE_SUITE_NON,
        'corri_air': RoomType.MALE_CORRI_AIR,
        'corri_non': RoomType.MALE_CORRI_NON,
    },
}

mixed_room_pref_to_type = {
    'corri_air': defaultdict(lambda: RoomType.MIXED_CORRI_AIR),
    'corri_non': defaultdict(lambda: RoomType.MIXED_CORRI_NON),
    'suite_air': {
        'Male': RoomType.MIXED_MALE_SUITE_AIR,
        'Female': RoomType.MIXED_FEMALE_SUITE_AIR,
    },
    'suite_non': {
        'Male': RoomType.MIXED_MALE_SUITE_NON,
        'Female': RoomType.MIXED_FEMALE_SUITE_NON,
    },
}

def get_empty_room_types_count(total_num_rooms):
    room_types = Counter()
    for room_type in RoomType:
        room_types[room_type] = 0
        
    room_types[RoomType.NO_ROOM] = total_num_rooms
    return room_types

def get_floor_stats(has_rf=False, has_laundry=False):
    stats = Counter()
    stats['corri_air'] = 6
    stats['corri_non'] = 12
    stats['suite_air'] = 6
    stats['suite_non'] = 12
    
    if has_rf or has_laundry:
        stats['suite_air'] -= 6
    
    if has_laundry:
        stats['corri_non'] -= 3
        stats['suite_non'] -= 6

    return stats

In [12]:
def get_room_stats(floor_counts):
    room_stats = dict()
    for gender in floor_counts.keys():
        room_stats[gender] = Counter()
        rf_count = floor_counts[gender]['rf']
        laundry_count = floor_counts[gender]['laundry']
        normal_count = floor_counts[gender]['normal']
        room_stats[gender] += reduce(lambda x, y: x + y, map(lambda x: get_floor_stats(has_rf=True), range(rf_count))) if rf_count > 0 else Counter()
        room_stats[gender] += reduce(lambda x, y: x + y, map(lambda x: get_floor_stats(has_laundry=True), range(laundry_count))) if laundry_count > 0 else Counter()
        room_stats[gender] += reduce(lambda x, y: x + y, map(lambda x: get_floor_stats(), range(normal_count))) if normal_count > 0 else Counter()
    
    return room_stats

def get_room_type_count_iterator(room_stats):
    room_type_counts = get_empty_room_types_count(len(data))
    for room_pref in room_stats['Male']:
        room_type = room_pref_to_type_map['Male'][room_pref]
        room_type_counts[room_type] += room_stats['Male'][room_pref]

    for room_pref in room_stats['Female']:
        room_type = room_pref_to_type_map['Female'][room_pref]
        room_type_counts[room_type] += room_stats['Female'][room_pref]
    
    for room_pref in room_stats['Mixed']:
        if room_pref in ['corri_air', 'corri_non', 'no_pref']:
            room_type = mixed_room_pref_to_type[room_pref]['mixed']
            room_type_counts[room_type] += room_stats['Mixed'][room_pref]

    num_suite_air = room_stats['Mixed']['suite_air'] / 6
    num_suite_non = room_stats['Mixed']['suite_non'] / 6

    room_type_counts_freeze = deepcopy(room_type_counts)

    for num_male_suite_air in range(num_suite_air + 1):
        num_female_suite_air = num_suite_air - num_male_suite_air

        for num_male_suite_non in range(num_suite_non + 1):
            num_female_suite_non = num_suite_non - num_male_suite_non

            room_type_counts = deepcopy(room_type_counts_freeze)
            room_type_counts[RoomType.MIXED_MALE_SUITE_AIR] += num_male_suite_air * 6
            room_type_counts[RoomType.MIXED_MALE_SUITE_NON] += num_male_suite_non * 6
            room_type_counts[RoomType.MIXED_FEMALE_SUITE_AIR] += num_female_suite_air * 6
            room_type_counts[RoomType.MIXED_FEMALE_SUITE_NON] += num_female_suite_non * 6
            yield room_type_counts

In [13]:
def get_valid_room_types(person):
    valid_rts = set()
    valid_rts.update(room_pref_to_type_map[person.gender].values())
    if person.gender_pref != 'mixed':
        return valid_rts
    
    for room_pref in mixed_room_pref_to_type:
        valid_rts.add(mixed_room_pref_to_type[room_pref][person.gender])

    return valid_rts

In [14]:
# constants to mark when people were assigned rooms
INITIAL, PREF_1, PREF_2, PREF_3, FIX_NO_PREF_ROOM, MOVE_MIXED_TO_SINGLE = range(6)

def get_assignment(room_type_count):
    rtc_left = deepcopy(room_type_count)
    assignments = [RoomType.NO_ROOM] * len(data)
    when_assigned = [INITIAL] * len(data)
    
    def assign_pref(pref_num):
        for idx, person in data.iterrows():
            if assignments[idx] is not RoomType.NO_ROOM:
                break
                
            if pref_num == 1:
                pref = person.room_pref_1
            elif pref_num == 2:
                pref = person.room_pref_2
            elif pref_num == 3:
                pref = person.room_pref_3
                
            if pref == 'no_pref':
                assignments[idx] = 'no_pref'
                when_assigned[idx] = pref_num
                continue
                
            if person.gender_pref == 'mixed':
                rt = mixed_room_pref_to_type[pref][person.gender]
                if rtc_left[rt] > 0:
                    assignments[idx] = rt
                    when_assigned[idx] = pref_num
                    rtc_left[rt] -= 1
                    continue

            rt = room_pref_to_type_map[person.gender][pref]
            if rtc_left[rt] > 0:
                assignments[idx] = rt
                when_assigned[idx] = pref_num
                rtc_left[rt] -= 1
                continue            
    
    def fix_no_prefs_and_no_room():
        for idx, person in data.iterrows():
            if assignments[idx] not in [RoomType.NO_ROOM, 'no_pref']:
                continue

            valid_rts = get_valid_room_types(person)

            for rt in valid_rts:
                if rtc_left[rt] < 1:
                    continue
                
#                 print 'yay! allocated empty room'
                assignments[idx] = rt
                when_assigned[idx] = FIX_NO_PREF_ROOM
                rtc_left[rt] -= 1
                break
            
            
            # if person doesn't have a room after looking at
            # all options, then we have to move on to the next
            # person.
            #
            # however, if they don't have a room but their pref
            # is no_pref, we should swap them with someone below.
            # because, they were more flexible in their options, 
            # they will be preferred over people who don't have
            # a room.
            if assignments[idx] != 'no_pref':
                continue
            
            p_assg_t = when_assigned[idx]
            for other_idx in range(len(assignments)):
                other_assg = assignments[other_idx]
                other_assg_t = when_assigned[other_idx]
                if other_assg_t >= p_assg_t and other_assg in valid_rts:
                    assignments[other_idx], assignments[idx] = assignments[idx], assignments[other_idx]
                    when_assigned[other_idx] = when_assigned[idx] = FIX_NO_PREF_ROOM
        return
    
    def move_mixed_to_single():
        for idx, person in data.iterrows():
            if assignments[idx] is not None:
                break
        
            if person.gender_pref == 'single':
                break
            
            for other_idx in reversed(data.index):
                if assignments[idx] is not None:
                    break

                other_gender = data.gender[other_idx]
                if other_gender == person.gender:
                    continue
                
                single_gender_rts = room_pref_to_type_map[other_gender].values()
                single_gender_rts.remove(RoomType.NO_PREF)
                for rt in single_gender_rts:
                    if rtc_left[rt] < 1:
                        continue
                    
                    # fix the when_assigned
                    print 'yay! swapping mixed to single'
                    assignments[idx] = assignments[other_idx]
                    assignments[other_idx] = rt
                    rtc_left[rt] -= 1
                    break                
                
    assign_pref(1)
    assign_pref(2)
    assign_pref(3)
    fix_no_prefs_and_no_room()
    move_mixed_to_single()
    
    return assignments

# Algo Driver

In [15]:
@timeit
def get_possibilities():
    room_type_counts = []
    floor_counts = []
    fcs = list(get_num_floor_types_iterator())
    for floor_count in tqdm(fcs):
        room_stats = get_room_stats(floor_count)
        rtcs = list(get_room_type_count_iterator(room_stats))
        room_type_counts.extend(rtcs)
        floor_counts.extend([floor_count] * len(rtcs))

    assignments = Parallel(n_jobs=-1)(delayed(get_assignment)(rtc) for rtc in tqdm(room_type_counts[:1]))
    return assignments, room_type_counts, floor_counts

assgs, rtcs, fcs = get_possibilities()

  0%|          | 2/1080 [00:00<01:13, 14.66it/s]

'get_num_floor_types_iterator' 		 0.0000 s


100%|██████████| 1/1 [00:00<00:00,  3.21it/s]

'get_possibilities' 		 9.1517 s


In [19]:
none_count = map(lambda x: x.count(RoomType.NO_ROOM), tqdm(assgs))
min_none_count = min(none_count)
min_none_count_index = none_count.index(min_none_count)
print '{} rooms will be empty @ configuration {}'.format(min_none_count, min_none_count_index)
rtc = rtcs[min_none_count_index]
ass = get_assignment(rtc)
rtc - Counter(ass)

100%|██████████| 1/1 [00:00<00:00, 2369.66it/s]

68 rooms will be empty @ configuration 0


Counter({<RoomType.MIXED_FEMALE_SUITE_AIR: 10>: 20,
         <RoomType.MIXED_FEMALE_SUITE_NON: 11>: 45,
         <RoomType.NO_ROOM: 14>: 298})

In [17]:
fcs[min_none_count_index]

{'Female': {'laundry': 0, 'normal': 0, 'rf': 0},
 'Male': {'laundry': 0, 'normal': 0, 'rf': 0},
 'Mixed': {'laundry': 1, 'normal': 7, 'rf': 3}}

In [18]:
data['ass'] = ass
data.head(50)

,gender,room_pref_1,room_pref_2,room_pref_3,gender_pref,ass
0,Female,corri_air,no_pref,no_pref,mixed,RoomType.MIXED_CORRI_AIR
1,Female,suite_air,suite_non,corri_air,mixed,RoomType.MIXED_FEMALE_SUITE_AIR
2,Female,corri_air,corri_non,no_pref,single,RoomType.NO_ROOM
3,Female,suite_air,suite_non,corri_air,mixed,RoomType.MIXED_FEMALE_SUITE_AIR
4,Female,corri_non,corri_air,suite_non,single,RoomType.NO_ROOM
5,Male,corri_air,no_pref,no_pref,single,RoomType.NO_ROOM
6,Male,corri_air,suite_air,no_pref,single,RoomType.NO_ROOM
7,Female,suite_air,suite_non,corri_air,mixed,RoomType.MIXED_FEMALE_SUITE_AIR
8,Female,corri_air,corri_non,suite_non,single,RoomType.NO_ROOM
9,Female,suite_non,corri_non,no_pref,mixed,RoomType.MIXED_FEMALE_SUITE_NON
